In [1]:
from _global_vars import *
from clean import get_outliers_df, remove_outliers

22/11/22 17:23:30 WARN Utils: Your hostname, LAPTOP-VAB0S7AL resolves to a loopback address: 127.0.1.1; using 172.31.199.143 instead (on interface eth0)
22/11/22 17:23:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/22 17:23:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# read in cleaned df
sdf = spark.read.parquet('../data/curated/yt2019_feature_eng.parquet')
#sdf.show(1, vertical=True, truncate=100)

In [3]:
# Note current record count
sdf.count()

7692971

---
## Erroneous record cleaning
---

### Filters (reflective of findings from univariate/preliminary analysis)
* Restrict range of passenger_count (1:6) [drop other records]
* Only take 312miles > trip distances > 0 (3 hours at maximum speed)
* Fares capped at $500 (0:500)
* remove trips with average speed over 104 mph and remove trips less than 1 mph
* Limit trip times to under 5 hours (300 minutes) (more than it takes to drive entire perimeter of nyc)
* Trips with a time of less than 2 minutes should be excluded => not very realistic => passenger may have cancelled
* Congestion surcharge capped at 10 usd -> any records beyond this can go away
* Ratecode == 1 and fare < 2.5

In [4]:
# get & inspect outlier breakdown from helper function
outlier_df = get_outliers_df(sdf)
print(outlier_df.sort_values('prop_of_total', ascending=False))

                        0  prop_of_total
congestion        4884448       0.634923
ratecode_min_fee   273595       0.035564
t_time             209885       0.027283
DOLocation         165931       0.021569
PULocationID       163645       0.021272
passenger_count    146056       0.018986
t_speed             85634       0.011131
vendorid            76890       0.009995
pay_type            72971       0.009485
trip_distance       55016       0.007151
rcode_id            28917       0.003759
fare_amount          9797       0.001274
tip_amount            105       0.000014


In [5]:
# apply outlier detection to df
sdf = remove_outliers(sdf)
sdf.groupBy("is_valid_record").count().show()

+---------------+-------+
|is_valid_record|  count|
+---------------+-------+
|           true|2541382|
|          false|5151589|
+---------------+-------+



In [6]:
# remove bad records
sdf_filtered = sdf.filter(sdf.is_valid_record == True)
sdf_filtered.count()

2541382

In [7]:
# inspect summary statistics
summary_stats_continuous = sdf_filtered.describe(*non_categorical_features)
summary_stats_continuous.show()

+-------+------------------+------------------+--------------------+------------------+-----------------+
|summary|        tip_amount|     trip_distance|congestion_surcharge|   passenger_count|      fare_amount|
+-------+------------------+------------------+--------------------+------------------+-----------------+
|  count|           2541382|           2541382|             2541382|           2541382|          2541382|
|   mean|1.7463542316735217| 2.434506898215296|3.049521874318776...| 1.588333040841558|11.23601370435456|
| stddev|1.9553641075454762|2.7526498333644738|0.008731388600784333|1.2225514988918524|7.972077734229534|
|    min|               0.0|              0.04|                 0.0|               1.0|              2.5|
|    max|             444.8|              52.1|                 2.5|               6.0|            169.0|
+-------+------------------+------------------+--------------------+------------------+-----------------+



In [8]:
sdf_filtered.write.mode('overwrite').parquet(export_relative_dir + "yt2019_cleaned.parquet")

22/11/22 17:23:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
spark.sql("CLEAR CACHE")
spark.stop()